## 0.前置内容

super() 函数是用于调用父类(超类)的一个方法。

super() 是用来解决多重继承问题的，直接用类名调用父类方法在使用单继承的时候没问题，但是如果使用多继承，会涉及到查找顺序（MRO）、重复调用（钻石继承）等种种问题。

MRO 就是类的方法解析顺序表, 其实也就是继承父类方法时的顺序表。

### 语法
以下是 super() 方法的语法:

`super(type[, object-or-type])`

### 参数
type -- 类。
object-or-type -- 类，一般是 self

### 实例1

In [5]:
class A:
    def add(self,x):
        y = x + 1
        print(y)
class B(A):
    def add(self,x):
        super().add(x)
b = B()
b.add(2)

3


### 实例2

In [6]:
class  FooParent():
    def __init__(self):
        self.parent = 'I\'m the parent.'
        print('Parent')
    def bar(self,message):
        print('%s from Parent'%(message))
class FooChild(FooParent):
    def __init__(self):
        #super(FooChild,self) 首先找到 FooChild 的父类（就是FooParent类 ），
        #然后把 FooChild 类的对象转换为 FooParent 类的对象    
        super(FooChild,self).__init__()
        print ('Child')
    def bar(self,message):
        super(FooChild,self).bar(message)
        

在本节中，我们将介绍如何使用PyTorch更方便地实现线性回归的训练。

In [1]:
import torch
import random
import numpy as np
from matplotlib import pyplot as plt
import torch.nn as nn

## 1.生成数据集

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 2.读取数据

PyTorch提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用Data代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量。

In [3]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features,labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

这里data_iter的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本。

In [4]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.9278,  0.1635],
        [-0.4916,  0.4914],
        [ 0.6640,  0.4041],
        [-0.4478, -0.3045],
        [ 1.6311, -0.1686],
        [ 1.6250, -0.7376],
        [-1.6726,  1.2790],
        [ 0.3153, -2.0649],
        [-0.0450, -0.1067],
        [ 1.8729, -1.1401]]) tensor([ 1.7893,  1.5615,  4.1506,  4.3341,  8.0489,  9.9507, -3.4878, 11.8344,
         4.4784, 11.8286])


## 3.定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，PyTorch提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用PyTorch更简洁地定义线性回归。

首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了autograd，而nn就是利用autograd来定义模型。nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先来看看如何用nn.Module实现一个线性回归模型。

In [ ]:
class LinearNet(nn.Moudle):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        